#  Using UNet for vehicle detection

In [3]:
# Import libraries
import cv2
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
% matplotlib inline
import glob

from keras.models import Model
from keras.layers import Input, merge, Convolution2D, MaxPooling2D, UpSampling2D, Reshape, core, Dropout,Lambda
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as K
from scipy.ndimage.measurements import label

from moviepy.editor import VideoFileClip

# Some helper functions

In [4]:

img_rows = 640
img_cols = 960
heatmap_prev = np.zeros((img_rows,img_cols))
heatmap_n = [np.zeros((img_rows,img_cols))]*10

def smooth_heatmap(heatmap):
    # Smoothing heatmap as average of 10 previous frames
    global heatmap_n
    
    heatmap_1 = heatmap_n[1:]
    heatmap_1.append(heatmap) 
    heatmap_n = heatmap_1   
    heatmap = np.mean(heatmap_n,axis=0)
    
   
    return heatmap 


def pred_for_img(img):
    img = cv2.resize(img,(img_cols, img_rows))
    img = np.reshape(img,(1,img_rows, img_cols,3))
    pred = model.predict(img)
    return pred,img[0]

    
def get_labeled_bboxes(img, labels):
    # Get labeled boxex
    bbox_all = []
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        
        # Define a bounding box based on min/max x and y
        if ((np.max(nonzeroy)-np.min(nonzeroy)> 40) & (np.max(nonzerox)-np.min(nonzerox)> 40)):
            bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
            # Draw the box on the image       
            #cv2.rectangle(img, bbox[0], bbox[1], (0,0,255),6)
            bbox_all.append(bbox)
    # Return the image
    return bbox_all

def get_BB_new(img):
    # Take in RGB image
    pred,img = pred_for_img(img)
    img  = np.array(img,dtype= np.uint8)
    img_pred = np.array(255*pred[0],dtype=np.uint8)
    heatmap = img_pred[:,:,0]
    heatmap = smooth_heatmap(heatmap)
    #print(np.max(heatmap))
    heatmap[heatmap> 240] = 255
    heatmap[heatmap<=240] = 0    
    labels = label(heatmap)
    
    bbox_all = get_labeled_bboxes(np.copy(img), labels)
    return bbox_all

# Defining UNet Models

In [5]:
### IOU  coeff and loss calculation
smooth=1.0
def IOU_calc(y_true, y_pred):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    
    return 2*(intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def IOU_calc_loss(y_true, y_pred):
    return -IOU_calc(y_true, y_pred)

In [6]:
##### Image size, 
img_rows = 640
img_cols = 960

#img_rows = 320
#img_cols = 480

#img_rows=480
#img_cols=720

In [7]:
#based on repository https://github.com/orobix/retina-unet, 
#slightly modified to save GPU memory, original code in comments
def get_gnet(drop=0.0):
    inputs = Input((img_rows, img_cols,3))
    inputs_norm = Lambda(lambda x: x/127.5 - 1.)
  #  conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(inputs)  
    conv1 = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(inputs)  
    conv1 = Dropout(drop)(conv1)
  #  conv1 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv1)
    conv1 = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(conv1)
   # up1 = UpSampling2D(size=(2, 2))(conv1)
  
    conv2 = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(conv1)
    conv2 = Dropout(drop)(conv2)
    conv2 = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    #
    conv3 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(pool2)
    conv3 = Dropout(drop)(conv3)
    conv3 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    #
    conv4 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool3)
    conv4 = Dropout(drop)(conv4)
    conv4 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    #
    conv5 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool4)
    conv5 = Dropout(drop)(conv5)
    conv5 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv5)
    #
    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=3)
    conv6 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(up6)
    conv6 = Dropout(drop)(conv6)
    conv6 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv6)
    #
    up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=3)
    conv7 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(up7)
    conv7 = Dropout(drop)(conv7)
    conv7 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv7)
    #
    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=3)
    conv8 = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(up8)
    conv8 = Dropout(drop)(conv8)
    conv8 = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(conv8)
    #
  #  pool4 = MaxPooling2D(pool_size=(2, 2))(conv8)

  #  conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(pool4)
    conv9 = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(conv8)
    conv9 = Dropout(drop)(conv9)
 #   conv9 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv9)
    conv9 = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(conv9)

    #
    conv10 = Convolution2D(1, 1, 1, activation='sigmoid')(conv9)

    model = Model(input=inputs, output=conv10)

   
    return model



In [8]:
def get_adopted_unet():
    inputs = Input((img_rows, img_cols,3))
    inputs_norm = Lambda(lambda x: x/127.5 - 1.)
    conv1 = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(inputs)
    conv1 = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(pool1)
    conv2 = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(pool2)
    conv3 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(pool3)
    conv4 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)

    conv5 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(pool4)
    conv5 = Convolution2D(128, 3, 3, activation='relu', border_mode='same')(conv5)

    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=3)
    conv6 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(up6)
    conv6 = Convolution2D(64, 3, 3, activation='relu', border_mode='same')(conv6)

    up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=3)
    conv7 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(up7)
    conv7 = Convolution2D(32, 3, 3, activation='relu', border_mode='same')(conv7)

    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=3)
    conv8 = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(up8)
    conv8 = Convolution2D(16, 3, 3, activation='relu', border_mode='same')(conv8)

    up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=3)
    conv9 = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(up9)
    conv9 = Convolution2D(8, 3, 3, activation='relu', border_mode='same')(conv9)

    conv10 = Convolution2D(1, 1, 1, activation='sigmoid')(conv9)

    model = Model(input=inputs, output=conv10)

    
    return model


# Loading trained UNet model

In [9]:
##### Image size, 
img_rows = 640
img_cols = 960

#recreate model and load trained weights
model = get_gnet()
model.compile(optimizer=Adam(lr=1e-4), loss=IOU_calc_loss, metrics=[IOU_calc])
model.load_weights("model_Unet_Weights_640_960_e400.h5")   
   
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_1 (InputLayer)             (None, 640, 960, 3)   0                                            
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 640, 960, 8)   224         input_1[0][0]                    
____________________________________________________________________________________________________
dropout_1 (Dropout)              (None, 640, 960, 8)   0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 640, 960, 8)   584         dropout_1[0][0]                  
___________________________________________________________________________________________

In [10]:
def process_frame(image):   
    
    image_bb = np.copy(image)
    bbox_cars = get_BB_new(image_bb)
    result=image_bb
    img_res_shape = result.shape
    for bbox in bbox_cars:
        cv2.rectangle(result, 
                      (np.int32(bbox[0][0]*img_res_shape[1]/img_cols),
                       np.int32(bbox[0][1]*img_res_shape[0]/img_rows)), 
                      (np.int32(bbox[1][0]*img_res_shape[1]/img_cols),
                       np.int32(bbox[1][1]*img_res_shape[0]/img_rows)),(0,255,0),6)
    return result

In [11]:
heatmap_n = [np.zeros((img_rows,img_cols))]*5
video_input1 = 'test_video.mp4'
video_output1 = 'test_video_solution_gnet_400.mp4'
clip = VideoFileClip(video_input1)
white_clip = clip.fl_image(process_frame)  # NOTE: this function expects color images!
white_clip.write_videofile(video_output1, audio=False)

[MoviePy] >>>> Building video test_video_solution_gnet_400.mp4
[MoviePy] Writing video test_video_solution_gnet_400.mp4


 97%|█████████▋| 38/39 [00:03<00:00, 11.18it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_video_solution_gnet_400.mp4 



In [12]:
heatmap_n = [np.zeros((img_rows,img_cols))]*5
video_input1 = 'project_video.mp4'
video_output1 = 'project_video_solution_gnet_400.mp4'
clip = VideoFileClip(video_input1)
white_clip = clip.fl_image(process_frame)  # NOTE: this function expects color images!
white_clip.write_videofile(video_output1, audio=False)


[MoviePy] >>>> Building video project_video_solution_gnet_400.mp4
[MoviePy] Writing video project_video_solution_gnet_400.mp4


100%|█████████▉| 1260/1261 [01:50<00:00, 11.39it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_solution_gnet_400.mp4 



# Load trained adopted UNet

In [13]:
##### Image size, 
img_rows = 640
img_cols = 960

img_rows = 560
img_cols = 720



#recreate model and load trained weights
#del model
model = get_adopted_unet()
model.compile(optimizer=Adam(lr=1e-4), loss=IOU_calc_loss, metrics=[IOU_calc])

model.load_weights("model_AdoptedUnet_Weights_640_960_e500.h5")   


   
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 560, 720, 3)   0                                            
____________________________________________________________________________________________________
convolution2d_20 (Convolution2D) (None, 560, 720, 8)   224         input_2[0][0]                    
____________________________________________________________________________________________________
convolution2d_21 (Convolution2D) (None, 560, 720, 8)   584         convolution2d_20[0][0]           
____________________________________________________________________________________________________
maxpooling2d_4 (MaxPooling2D)    (None, 280, 360, 8)   0           convolution2d_21[0][0]           
___________________________________________________________________________________________

In [14]:
heatmap_n = [np.zeros((img_rows,img_cols))]*10
video_input1 = 'test_video.mp4'
video_output1 = 'test_video_solution_adopted_unet_500.mp4'
clip = VideoFileClip(video_input1)
white_clip = clip.fl_image(process_frame)  # NOTE: this function expects color images!
white_clip.write_videofile(video_output1, audio=False)

[MoviePy] >>>> Building video test_video_solution_adopted_unet_500.mp4
[MoviePy] Writing video test_video_solution_adopted_unet_500.mp4


 97%|█████████▋| 38/39 [00:01<00:00, 19.89it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: test_video_solution_adopted_unet_500.mp4 



In [15]:
heatmap_n = [np.zeros((img_rows,img_cols))]*10
video_input1 = 'project_video.mp4'
video_output1 = 'project_video_solution_adopted_unet_500.mp4'
clip = VideoFileClip(video_input1)
white_clip = clip.fl_image(process_frame)  # NOTE: this function expects color images!
white_clip.write_videofile(video_output1, audio=False)

[MoviePy] >>>> Building video project_video_solution_adopted_unet_500.mp4
[MoviePy] Writing video project_video_solution_adopted_unet_500.mp4


100%|█████████▉| 1260/1261 [01:07<00:00, 18.70it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_solution_adopted_unet_500.mp4 

